In [1]:

import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
from scipy.spatial.transform import Rotation as R 


In [ ]:
import open3d as o3d
print("Open3D Version:", o3d.__version__)
print("Open3D Path:", o3d.__file__)
print("Has Sphere class:", hasattr(o3d.geometry, 'Sphere'))
print("Has create_mesh_sphere function:", hasattr(o3d.geometry, 'create_mesh_sphere'))
print("Contents of o3d.geometry:", dir(o3d.geometry))
exit()

Open3D Version: 0.19.0
Open3D Path: d:\Miniconda\envs\SLAM\lib\site-packages\open3d\__init__.py
Has Sphere class: False
Has create_mesh_sphere function: False
Contents of o3d.geometry: ['All', 'Average', 'AxisAlignedBoundingBox', 'Color', 'DeformAsRigidAsPossibleEnergy', 'FilterScope', 'Gaussian3', 'Gaussian5', 'Gaussian7', 'Geometry', 'Geometry2D', 'Geometry3D', 'HalfEdge', 'HalfEdgeTriangleMesh', 'Image', 'ImageFilterType', 'KDTreeFlann', 'KDTreeSearchParam', 'KDTreeSearchParamHybrid', 'KDTreeSearchParamKNN', 'KDTreeSearchParamRadius', 'LineSet', 'MeshBase', 'Normal', 'Octree', 'OctreeColorLeafNode', 'OctreeInternalNode', 'OctreeInternalPointNode', 'OctreeLeafNode', 'OctreeNode', 'OctreeNodeInfo', 'OctreePointColorLeafNode', 'OrientedBoundingBox', 'PointCloud', 'Quadric', 'RGBDImage', 'SimplificationContraction', 'Smoothed', 'Sobel3dx', 'Sobel3dy', 'Spokes', 'TetraMesh', 'TriangleMesh', 'Vertex', 'Voxel', 'VoxelGrid', '__doc__', '__loader__', '__name__', '__package__', '__spec__', 'g

: 

In [2]:
# Set dataset path
DATASET_PATH = r'rgbd_dataset_freiburg3_long_office_household'  

fx = 525.0
fy = 525.0
cx = 319.5
cy = 239.5

K = np.array([[fx, 0, cx],
              [0, fy, cy],
              [0,  0,  1]])

In [3]:
from torch.utils.data import Dataset
import torch
class TUMDataset(Dataset):
    def __init__(self, rgb_file, depth_file, gt_file, transform=None, depth_transform=None, base_dir=""):
        rgb_file = os.path.join(base_dir,rgb_file)
        depth_file = os.path.join(base_dir,depth_file)
        gt_file = os.path.join(base_dir,gt_file)
        self.rgb_data = self._read_file(rgb_file)
        self.depth_data = self._read_file(depth_file)
        self.gt_data = self._read_file(gt_file)

        n = min(len(self.rgb_data), len(self.depth_data), len(self.gt_data))
        self.rgb_data = self.rgb_data[:n]
        self.depth_data = self.depth_data[:n]
        self.gt_data = self.gt_data[:n]

        self.transform = transform
        self.depth_transform = depth_transform
        self.base_dir = base_dir

    def _read_file(self, filename):
        with open(filename, 'r') as f:
            lines = [line.strip() for line in f if line.strip() and not line.startswith("#")]
        data = []
        for line in lines:
            parts = line.replace(",", " ").replace("\t", " ").split()
            timestamp = float(parts[0])
            data.append((timestamp, parts[1:]))
        return data

    def __len__(self):
        return len(self.rgb_data)

    def __getitem__(self, idx):
        rgb_path = os.path.join(self.base_dir, self.rgb_data[idx][1][0])
        depth_path = os.path.join(self.base_dir, self.depth_data[idx][1][0])
        pose_vals = list(map(float, self.gt_data[idx][1]))

        rgb = Image.open(rgb_path).convert("RGB")
        depth = Image.open(depth_path).convert("I")  # 'I' mode for 32-bit integer pixels

        if self.transform:
            rgb = self.transform(rgb)
        if self.depth_transform:
            depth = self.depth_transform(depth)
            if depth.ndim == 2:  # If shape is [H, W], add channel dimension
                depth = depth.unsqueeze(0)

        pose = torch.tensor(pose_vals, dtype=torch.float32)

        return {
            "rgb": rgb,
            "depth": depth,
            "pose": pose
        }

In [4]:
data = TUMDataset(
    rgb_file='rgb.txt',
    depth_file='depth.txt',
    gt_file='groundtruth.txt',
    base_dir=DATASET_PATH
)

In [5]:

class TUMTimeDataset(Dataset):
    def __init__(self, rgb_file, depth_file, transform=None, depth_transform=None, base_dir=""):
        self.base_dir = base_dir

        # Read raw data for each stream (excluding ground truth)
        self.rgb_data_raw = self._read_file(os.path.join(base_dir, rgb_file))
        self.depth_data_raw = self._read_file(os.path.join(base_dir, depth_file))

        # Create a single list of all events, tagged by type and including filename
        self.event_list = []

        # Add RGB events
        for timestamp, paths in self.rgb_data_raw:
            # paths[0] is the relative path, e.g., "rgb/1305031441.946399.png"
            self.event_list.append((timestamp, 'rgb', paths[0], os.path.basename(paths[0])))

        # Add Depth events
        for timestamp, paths in self.depth_data_raw:
            # paths[0] is the relative path, e.g., "depth/1305031441.949704.png"
            self.event_list.append((timestamp, 'depth', paths[0], os.path.basename(paths[0])))

        # Sort the entire list of events by timestamp
        self.event_list.sort(key=lambda x: x[0])

        # Find the minimum starting time and normalize all timestamps
        if self.event_list:
            min_overall_timestamp = self.event_list[0][0]
            self.event_list = [
                (timestamp - min_overall_timestamp, event_type, relative_path, filename)
                for timestamp, event_type, relative_path, filename in self.event_list
            ]
        else:
            min_overall_timestamp = 0.0 # No data
            print("Warning: No RGB or Depth data found in the files.")


        self.transform = transform
        self.depth_transform = depth_transform
        print(f"Dataset loaded. Total {len(self.event_list)} chronological events (RGB and Depth only).")
        print(f"Timestamps normalized. First event at time 0.0.")


    def _read_file(self, filename):
        with open(filename, 'r') as f:
            lines = [line.strip() for line in f if line.strip() and not line.startswith("#")]
        data = []
        for line in lines:
            parts = line.replace(",", " ").replace("\t", " ").split()
            timestamp = float(parts[0])
            # For RGB/Depth, parts[1:] will be just the path.
            data.append((timestamp, parts[1:]))
        return data

    def __len__(self):
        return len(self.event_list)

    def __getitem__(self, idx):
        # Unpack event_data to get relative_path and filename
        normalized_timestamp, event_type, relative_path, filename = self.event_list[idx]

        result = {
            "timestamp": normalized_timestamp,
            "event_type": event_type,
            "filename": filename # Include the filename in the result
        }

        if event_type == 'rgb':
            rgb_full_path = os.path.join(self.base_dir, relative_path)
            rgb = Image.open(rgb_full_path).convert("RGB")
            if self.transform:
                rgb = self.transform(rgb)
            result["rgb"] = rgb
        elif event_type == 'depth':
            depth_full_path = os.path.join(self.base_dir, relative_path)
            depth = Image.open(depth_full_path).convert("I")  # 'I' mode for 32-bit integer pixels

            if self.depth_transform:
                depth = self.depth_transform(depth)
                # Ensure depth has a channel dimension if needed (e.g., for PyTorch models)
                if isinstance(depth, torch.Tensor) and depth.ndim == 2:
                    depth = depth.unsqueeze(0)
            result["depth"] = depth

        return result

In [43]:
timed_data = TUMTimeDataset(
    rgb_file='rgb.txt',
    depth_file='depth.txt',
    base_dir=DATASET_PATH,
    # transform=rgb_transform,
    # depth_transform=depth_transform
)

Dataset loaded. Total 5094 chronological events (RGB and Depth only).
Timestamps normalized. First event at time 0.0.


In [44]:
def pose_to_T_matrix(pose_vec):
    """
    Converts a pose vector [x, y, z, qx, qy, qz, qw] to a 4x4 homogeneous transformation matrix.
    """
    t = pose_vec[:3]
    q = pose_vec[3:]
    rot_matrix = R.from_quat(q).as_matrix() # scipy.spatial.transform.Rotation handles [x,y,z,w] or [w,x,y,z] based on input
    T = np.eye(4)
    T[:3, :3] = rot_matrix
    T[:3, 3] = t
    return T


In [45]:

def T_matrix_to_pose(T_matrix):
    """
    Converts a 4x4 homogeneous transformation matrix to a pose vector [x, y, z, qx, qy, qz, qw].
    """
    t = T_matrix[:3, 3]
    rot_matrix = T_matrix[:3, :3]
    q = R.from_matrix(rot_matrix).as_quat() # Returns [x,y,z,w]
    return np.concatenate((t, q))


In [46]:
def unproject_2d_to_3d_depth(u, v, depth_value, K_inv):
    """
    Unprojects a 2D pixel (u,v) with a depth value to a 3D point in camera coordinates.
    K_inv is the inverse of the camera intrinsic matrix.
    """
    if depth_value <= 0: # Invalid depth
        return None
    # Normalized image coordinates
    x_c = (u - K[0,2]) / K[0,0] # (u - cx) / fx
    y_c = (v - K[1,2]) / K[1,1] # (v - cy) / fy
    # 3D point in camera coordinates
    P_c = np.array([x_c * depth_value, y_c * depth_value, depth_value])
    return P_c


In [47]:

def project_3d_to_2d(P_3d_cam, K):
    """
    Projects a 3D point (P_3d_cam) in camera coordinates to 2D pixel coordinates.
    Returns (u, v).
    """
    if P_3d_cam[2] <= 0: # Point behind camera or at infinity
        return None
    u = K[0,0] * (P_3d_cam[0] / P_3d_cam[2]) + K[0,2]
    v = K[1,1] * (P_3d_cam[1] / P_3d_cam[2]) + K[1,2]
    return u, v

# --- 4. EKF Core Functions (Placeholders for complex math) ---


In [48]:

def predict_state(current_pose, current_covariance, delta_t, Q_process_noise):
    """
    Predicts the next state and covariance based on a motion model.
    This is a simplified constant velocity model. For real EKF, needs Jacobians.
    State: [x, y, z, qx, qy, qz, qw]
    """
    # Simple prediction: assume no change in pose, but uncertainty grows
    # In a real EKF:
    # 1. Augment state with velocity: [x,y,z,qx,qy,qz,qw,vx,vy,vz,wx,wy,wz]
    # 2. Propagate x based on velocity and delta_t:
    #    predicted_x = f(current_pose, current_velocity, delta_t)
    # 3. Propagate P using Jacobian F: predicted_P = F @ current_P @ F.T + Q
    # This example only models growing uncertainty.
    predicted_pose = current_pose.copy()

    # Add process noise (Q_process_noise) to the covariance
    # For a proper EKF, Q depends on delta_t and actual noise parameters.
    predicted_covariance = current_covariance + Q_process_noise * delta_t # Simplistic noise growth

    return predicted_pose, predicted_covariance


In [49]:

def update_state_ekf(predicted_state, predicted_covariance, measurement, H_jacobian, R_measurement_noise):
    """
    Performs the Extended Kalman Filter update step.
    predicted_state: State vector before measurement (x_k^-)
    predicted_covariance: Covariance matrix before measurement (P_k^-)
    measurement: The actual sensor measurement (z_k)
    H_jacobian: Measurement Jacobian (H_k) - relates state to measurement (h(x))
    R_measurement_noise: Covariance of the measurement noise (R_k)
    """
    # Calculate Kalman Gain (K_k)
    # y = z_k - h(x_k^-)  # Residual
    # S = H_k @ P_k^- @ H_k.T + R_k
    # K_k = P_k^- @ H_k.T @ np.linalg.inv(S)

    # Update state and covariance
    # updated_x = x_k^- + K_k @ y
    # updated_P = (I - K_k @ H_k) @ P_k^-

    # Placeholder: In a real implementation, this is where the core EKF math happens.
    # For now, just return the predicted state as updated.
    print("  (Placeholder: EKF update logic not implemented here)")
    return predicted_state, predicted_covariance
